In [4]:
cd ../..

/Users/leowalker/Library/Mobile Documents/com~apple~CloudDocs/Documents/elevated_ambitions


/Users/leowalker/Library/Mobile Documents/com~apple~CloudDocs/Documents/elevated_ambitions/.venv-explorer/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
!uv pip install devtools

Using Python 3.12.8 environment at: .venv-explorer
Resolved 5 packages in 10ms                                          
Uninstalled 1 package in 8ms
Installed 2 packages in 3ms                                 
 - asttokens==3.0.0
 + asttokens==2.4.1
 + devtools==0.12.2


In [10]:
from pymongo import MongoClient, DESCENDING
from dotenv import load_dotenv
import os
from pprint import pprint
from backend.models.job_description_models import JobDescription
from devtools import debug  # Optional: for even prettier printing

# Load environment variables
load_dotenv()

# Connect to MongoDB
uri = os.getenv('MONGODB_URI')
client = MongoClient(uri)

# Get database and collection
db = client['jobs_db']
elevated_jobs = db['elevated_jobs']
original_jobs = db['job_listings']


In [14]:
# Find 3 most recent jobs
recent_jobs = elevated_jobs.find(
    {"created_at": {"$exists": True}},  # Ensure created_at exists
    sort=[("created_at", DESCENDING)]   # Sort by created_at in descending order
).limit(3)

# Convert to list and display
for i, job_data in enumerate(recent_jobs, 1):
    print(f"\n{'='*50}")
    print(f"Job {i} (Created: {job_data.get('created_at')})")
    print(f"{'='*50}")
    
    # Just print the structured_job field
    if 'structured_job' in job_data:
        pprint(job_data['structured_job'])
    else:
        print("No structured_job field found")
        print("Available fields:", list(job_data.keys()))


Job 1 (Created: 2024-12-27 19:47:51.050000)
{'metadata': {'job_id': None, 'source_url': None, 'date_posted': None, 'apply_link': 'https://www.linkedin.com/jobs/view/customer-partner-solutions-engineer-3p-google-identity-at-google-4095282410?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic', 'source_platform': 'LinkedIn'}, 'company_overview': {'company_name': 'Google', 'about': 'Google creates products and services that make the world a better place, and gTech’s role is to help bring them to life.', 'mission_and_values': 'Our mission is to advocate for Google’s users by creating helpful and trusted experiences across the product ecosystem.', 'size': None, 'industry': None, 'locations': 'Sunnyvale, CA'}, 'role_summary': {'title': 'Customer & Partner Solutions Engineer, 3P Google Identity', 'job_level': None, 'role_type': None, 'employment_type': 'Full-time', 'remote_options': None, 'team_or_department': None}, 'responsibilities_and_qualifications': {'respon

In [22]:
# Get the created_at date of the last job from previous query
last_job = elevated_jobs.find(
    {"created_at": {"$exists": True}},
    sort=[("created_at", DESCENDING)]
).limit(1).skip(9).next()  # Get the 10th job

last_date = last_job['created_at']

# Now find jobs created before this date
older_jobs = elevated_jobs.find(
    {
        "created_at": {
            "$exists": True,
            "$lt": last_date  # Find jobs created before last_date
        }
    },
    sort=[("created_at", DESCENDING)]
).limit(10)

# Display jobs
for i, job_data in enumerate(older_jobs, 1):
    print(f"\n{'='*50}")
    print(f"Job {i} (Created: {job_data.get('created_at')})")
    print(f"{'='*50}")
    
    if 'structured_job' in job_data:
        job = JobDescription(**job_data['structured_job'])
        print(job.model_dump_json(indent=2))
    else:
        print("No structured_job field found")
        print("Available fields:", list(job_data.keys()))


Job 1 (Created: 2024-12-27 19:46:39.460000)
{
  "metadata": {
    "job_id": "675b69713bbe7eccd30f4088",
    "source_url": "https://www.weareroku.com/jobs/solutions-engineer-roku-experiences-san-jose-california-united-states?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic",
    "date_posted": null,
    "apply_link": "https://www.weareroku.com/jobs/solutions-engineer-roku-experiences-san-jose-california-united-states?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic",
    "source_platform": "Roku Jobs"
  },
  "company_overview": {
    "company_name": "Roku",
    "about": "Roku is changing how the world watches TV. Roku is the #1 TV streaming platform in the US and Mexico, and we've set our sights on powering every television in the world.",
    "mission_and_values": "Roku is committed to offering a diverse range of benefits as part of our compensation package to support our employees and their families.",
    "size": null,
    